## Atelier 1 - Introduction à la variance de blocs

### 🎯 But pédagogique
Montrer comment la dépendance spatiale s'atténue avec la taille du support (**effet de support**).

---

### 🎓 Objectif

Montrer l’effet de la taille de support (bloc) sur la variance, comparée à :

- la **variance ponctuelle** ;
- la **variance de bloc théorique** (calculée à partir de l'intégrale de la fonction de covariance) ;
- la **variance de bloc expérimentale**, obtenue :
  - par **échantillonnage aléatoire** ;
  - ou par **moyenne sur sous-blocs**.

---

### 🔍 Concepts clés
- **Effet de support** : réduction de la variance lorsque la taille du bloc augmente.
- **Covariance spatiale** : mesure de la dépendance entre valeurs en fonction de la distance.
- **Variance de bloc** : mesure de la variabilité moyenne sur une surface ou un volume donné.

---



In [48]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
from numpy.fft import fft2, ifft2, fftshift
from scipy.stats import norm

def normal_score_transform(data):
    data = np.asarray(data).ravel()
    n = len(data)
    
    # Trier les données et garder les indices originaux
    sorted_idx = np.argsort(data)
    sorted_data = data[sorted_idx]
    
    # Quantiles empiriques (plotting positions)
    probs = (np.arange(1, n+1) - 0.5) / n  # méthode moyenne
    
    # Valeurs normales correspondantes
    norm_scores = norm.ppf(probs)
    
    # Tableau pour reconstruire dans l'ordre original
    result = np.empty_like(data, dtype=float)
    result[sorted_idx] = norm_scores
    
    return result
    
# --- Modèles de covariance ---
def spherical_covariance(h, range_, sill):
    cov = np.zeros_like(h)
    mask = h <= range_
    hr = h[mask] / range_
    cov[mask] = sill * (1 - 1.5 * hr + 0.5 * hr**3)
    return cov

def exponential_covariance(h, range_, sill):
    return sill * np.exp(-3 * h / range_)

def gaussian_covariance(h, range_, sill):
    return sill * np.exp(-3 * (h / range_)**2)
    
def get_covariance_model(model_name):
    if model_name == 'Sphérique':
        return spherical_covariance
    elif model_name == 'Exponentiel':
        return exponential_covariance
    elif model_name == 'Gaussien':
        return gaussian_covariance
    else:
        raise ValueError("Modèle inconnu")

def anisotropic_distance(X, Y, range_x, range_y, angle_deg=0):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    
    # Rotation + mise à l'échelle inverse des portées
    X_rot = (X * cos_a + Y * sin_a) / range_x
    Y_rot = (-X * sin_a + Y * cos_a) / range_y
    
    return np.sqrt(X_rot**2 + Y_rot**2)

# --- Génération FFT-MA anisotrope ---
def fftma_simulation(size, range_x, range_y, sill, nugget, angle_deg, model_name, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)

    h_aniso = anisotropic_distance(X, Y, range_x, range_y, angle_deg)
    cov_func = get_covariance_model(model_name)
    cov_model = cov_func(h_aniso, range_=1.0, sill=sill)
    cov_model += nugget * (X == 0) * (Y == 0)  # effet de pépite

    cov_fft = fft2(fftshift(cov_model))
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))

    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

# --- Agrégation par blocs carrés ---
def aggregate(field, block_size):
    if block_size == 0:
        return field
    s = field.shape[0]
    reduced_size = s // block_size
    # Découper le champ en blocs : (reduced_size, block_size, reduced_size, block_size)
    reshaped = field[:reduced_size*block_size, :reduced_size*block_size].reshape(
        reduced_size, block_size, reduced_size, block_size)
    # Moyenne sur les dimensions des blocs
    aggregated = reshaped.mean(axis=(1, 3))
    return aggregated

# --- Variance expérimentale des blocs ---
def variance_blocks(field, max_block_size):
    s = field.shape[0]
    variances = []
    block_sizes = list(range(0, max_block_size + 1, 2))
    for bsize in block_sizes:
        agg = aggregate(field, bsize)
        variances.append(np.var(agg))
    return block_sizes, np.array(variances)

# --- Variance théorique d'un bloc carré dans modèle sphérique ---
def theoretical_block_variance(range_x, range_y, sill, nugget, block_size, pixel_size=1, angle_deg=0, model_name = 'Sphérique'):
    if block_size == 0:
        return sill + nugget

    # Discrétisation du bloc [0, block_size] x [0, block_size]
    n = int(block_size / pixel_size)
    x = np.linspace(0, block_size, n, endpoint=False)
    X, Y = np.meshgrid(x, x)

    # Coordonnées en 2D pour tous les points
    points = np.stack([X.ravel(), Y.ravel()], axis=1)

    # Calcul de la matrice des distances anisotropes entre tous les points du bloc
    n_pts = len(points)
    dx = points[:, 0].reshape(-1, 1) - points[:, 0].reshape(1, -1)
    dy = points[:, 1].reshape(-1, 1) - points[:, 1].reshape(1, -1)
    h_aniso = anisotropic_distance(dx, dy, range_x, range_y, angle_deg)

    # Covariance entre tous les couples de points
    cov_func = get_covariance_model(model_name)
    cov = cov_func(h_aniso, range_=1.0, sill=sill)

    # Moyenne de la covariance (variance du bloc)
    return np.mean(cov) + nugget/(block_size*block_size)

# --- Paramètres fixes ---
size = 500           # taille du champ (500x500 pixels)
seed = 4263            # seed pour la simulation

# --- Fonction principale interactive ---
def interactive_variance(support, range_x, range_y, sill, nugget, angle_deg, model_name):
    if support > 50: support = 50
    
    field = fftma_simulation(
        size=size,
        range_x=range_x,
        range_y=range_y,
        sill=sill,
        nugget=nugget,
        angle_deg=angle_deg,
        model_name=model_name,
        seed=seed
    )
    field = normal_score_transform(field).reshape(field.shape)*np.sqrt(sill+nugget)
    
    plt.figure(figsize=(14,6))

    # Agrégation du champ
    agg = aggregate(field, support)

    # Calcul adaptatif de la légendre de couleur
    variance = sill + nugget
    std_dev = np.sqrt(variance)
    vmin = norm.ppf(0.05, loc=0, scale=std_dev)
    vmax = norm.ppf(0.95, loc=0, scale=std_dev)

    # Affichage du champ agrégé
    plt.subplot(1,2,1)
    plt.imshow(agg, cmap='viridis', origin='lower', vmin=vmin, vmax=vmax)
    plt.title(f'Champ agrégé, support = {support}x{support}')
    plt.colorbar(label='Valeurs')
    plt.axis('off')

    # Calcul variance expérimentale pour tous supports ≤ support max (50)
    max_support = 50
    bsizes, var_exp = variance_blocks(field, max_support)

    # Calcul variance théorique
    var_theo = [
        theoretical_block_variance(
            range_x=range_x, range_y=range_y, sill=sill,
            nugget = nugget, block_size=b, pixel_size=0.05*b,
            angle_deg=angle_deg, model_name = model_name
        )
        for b in bsizes
    ]

    # Affichage variance expérimentale vs théorique
    plt.subplot(1,2,2)
    plt.plot(bsizes, var_exp, 'o-', label='Variance expérimentale')
    plt.plot(bsizes, var_theo, 's--', label='Variance théorique')
    plt.axvline(support, color='red', linestyle=':', label=f'Support sélectionné = {support}')
    plt.xlabel('Taille du support (pixels)')
    plt.ylabel('Variance')
    plt.title('Variance des blocs vs taille de support')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.xlim(0, 50)
    plt.ylim(0, sill + nugget)
    plt.tight_layout()
    plt.show()

# --- Widgets ---
interact(
    interactive_variance,
    support=IntSlider(min=0, max=50, step=1, value=1, description='Support (pixels)'),
    range_x=FloatSlider(min=1, max=50, step=1, value=15, description='Portée X ($a_x)'),
    range_y=FloatSlider(min=1, max=50, step=1, value=15, description='Portée Y ($a_y)'),
    sill=FloatSlider(min=0.1, max=10, step=0.1, value=1.0, description='$c_1$'),
    nugget=FloatSlider(min=0, max=1, step=0.01, value=0, description='Effet de pépite ($c_0)'),
    angle_deg=FloatSlider(min=0, max=180, step=1, value=30, description='Angle ($θ$)'),
    model_name=Dropdown(options=['Sphérique', 'Exponentiel', 'Gaussien'], value='Sphérique', description='Modèle')
)

interactive(children=(IntSlider(value=1, description='Support (pixels)', max=50), FloatSlider(value=15.0, desc…

<function __main__.interactive_variance(support, range_x, range_y, sill, nugget, angle_deg, model_name)>